In [1]:
#Core libraries
import gpytorch
import torch
import numpy as np
import pandas as pd

#Ax library
from ax.modelbridge.generation_strategy import GenerationStep, GenerationStrategy
from ax.modelbridge.registry import Models
from botorch.models.gpytorch import GPyTorchModel
from botorch.utils.datasets import SupervisedDataset
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from ax.models.torch.botorch_modular.model import BoTorchModel
from ax.models.torch.botorch_modular.surrogate import Surrogate
from botorch.fit import fit_gpytorch_model
from gpytorch.mlls import ExactMarginalLogLikelihood, LeaveOneOutPseudoLikelihood, SumMarginalLogLikelihood
from gpytorch.mlls import VariationalELBO
from botorch.acquisition.monte_carlo import qExpectedImprovement
from botorch.acquisition import qNoisyExpectedImprovement
from botorch.acquisition.analytic import ExpectedImprovement, LogProbabilityOfImprovement
from ax.service.ax_client import AxClient, ObjectiveProperties
from botorch.models.gp_regression import FixedNoiseGP

#Deveper API
from ax import (
    ComparisonOp,
    ParameterType,
    RangeParameter,
    ChoiceParameter,
    FixedParameter,
    SearchSpace,
    Experiment,
    OutcomeConstraint,
    OrderConstraint,
    SumConstraint,
    OptimizationConfig,
    Objective,
    Metric,
    Runner,
    Data,
    Arm,
)
from ax.core import ObservationFeatures
#For Visualization Purposes
from ax.utils.notebook.plotting import render
from ax.modelbridge.cross_validation import cross_validate
from ax.plot.diagnostic import interact_cross_validation
from ax.plot.slice import plot_slice



[WARNING 01-31 12:56:31] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


## Get Data

In [2]:
#get modified features and train_final label from the csv file
# NAME_OF_FILE = 'modified_features.csv'
NAME_OF_FILE =  '/Users/jorgemedina/DOEFinal/DOE/AgML/Data/modified_features.csv'
mod_dataset = pd.read_csv(NAME_OF_FILE, index_col=0)
modified_features = mod_dataset.iloc[:,0:9] #the first ones are the features, the last one is the label
train_final_label = mod_dataset.iloc[:,9]
#transform to torch
train_final_label_pt = torch.tensor(train_final_label.values, dtype=torch.float)
modified_features_pt = torch.tensor(modified_features.values, dtype=torch.float)

print("You have a total of ", len(modified_features), " data points")
print("With ", len(modified_features.columns), " number of features each")
#print(" and ", len() , " label(s) each") # In theory you could want to do multi-objective optimization



############################################################################################################

modified_features.head() # What are the columns? That might be good to have it in the .csv file

You have a total of  51  data points
With  9  number of features each


,0,1,2,3,4,5,6,7,8
0,18.8,14.04,0.175,0.25,0.0060,7.8,1.25,60.0,25.0
1,18.8,14.04,0.175,0.25,0.0060,7.8,2.50,60.0,25.0
2,18.8,14.04,0.175,0.25,0.0060,7.8,5.00,60.0,25.0
3,18.8,14.04,0.175,0.25,0.0060,7.8,10.00,60.0,25.0
4,18.8,14.04,0.175,0.25,0.0007,3.5,0.25,60.0,25.0


## Define your models with GPyTorch-BoTorch

In [3]:
### This is how i figure base on a lot of conversation and trial and error that you can 
### initialize your model classes for use with Ax.dev framework. (e.g. speficying the likelihood directly in the __init__)
class ExactGPModel(gpytorch.models.ExactGP, GPyTorchModel):
    
    _num_outputs = 1 

    def __init__(self, train_X, train_Y,**kwargs):
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood(), **kwargs)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=9))
        self.to(train_X)
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

#matern models

class BaseGPMatern(gpytorch.models.ExactGP, GPyTorchModel):
    _num_outputs = 1 
    
    def __init__(self, train_X, train_Y, **kwargs):
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood(), **kwargs)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5,ard_num_dims=9))
        self.to(train_X)
            
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

# Ax.dev ingredients
Either Service or Developer API work for our purposes. Choose the one you prefer, Developer gives more flexibility to the creation of the optimization loop

## Using the Service API

### Generation Strategy

In [4]:
# Because we already have a warm start with raw data, we dont need SOBOL sampling to begin with. So we want to use BoTorch w qExpectedImprovement for sampling
# the mll_class can be either ExactMarginalLogLikelihood or LeaveOneOutPseudoLikelihood for our ExactGP models. The later is more stable.
gs = GenerationStrategy(
    steps=[
        GenerationStep(
            model=Models.BOTORCH_MODULAR,
            num_trials=-1,  # No limitation on how many trials should be produced from this step
            # For `BOTORCH_MODULAR`, we pass in kwargs to specify what surrogate or acquisition function to use.
            model_gen_kwargs={"max_retries": 10},
            model_kwargs={
                "surrogate":Surrogate(ExactGPModel,
                                      mll_class=LeaveOneOutPseudoLikelihood),
                
                "botorch_acqf_class": qExpectedImprovement,
                },
        ),
    ]
)

### SearchSpace


In [5]:
## Points to consider
#for the "choice" variables, add ordered = False 
# as parameter if you prefer one-hot encoding rather than ordinal encoding
# for more options look at Ax documentation on parameters and Trasformations (Transforms)
ax_parameters = [
    {
        "name": "Dimension1",
        "type": "range",
        "bounds": [12.0000,51.00000],
        "value_type":'float'
    },
    {
        "name": "Dimension2",
        "type": "range",
        "bounds": [6,26],
        "value_type": 'float'
    },
    {
        "name": "Dilution(OD)",
        "type": "range",
        "bounds": [0.12,0.44]
    },
    {   "name":"Nano_stock_vol",
        "type":"choice",
        "values": [0.25,0.5],

    },
    {
        "name": "Ag_acet_conc",
        "type": "choice",
        "values": [0.0007,0.006],

    },
     {
        "name": "Ag_acet_mass",
        "type": "choice",
        "values": [3.5,7.8,14.0],

     },
    {
        "name": "Doping%",
        "type": "range",
        "bounds": [0.1,10.0],
        "value_type": 'float'

    },
    {
        "name": "Time",
        "type": "choice",
        "values": [10.0,60.0,180.0],
        'value_type': 'float'
    },
    {
        "name": "Temperature",
        "type": "choice",
        "values": [0.0,25.0,50.0],
        "value_type": 'float',

    },
]



### Initialize the AxClient for the service api

In [6]:
ax_client = AxClient(generation_strategy = gs)
ax_client.create_experiment(parameters = ax_parameters, 
                            objectives= {"f":ObjectiveProperties(minimize=False)},)  #we want to maximze f, but it shouldnt matter
                                                                                     # f = QT*AUC2 

[INFO 01-31 12:56:47] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 01-31 12:56:47] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Dilution(OD). If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 01-31 12:56:47] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter Nano_stock_vol. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/Users/jorgemedina/miniforge3/envs/DOEv2/lib/python3.10/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "Nano_stock_vol". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this be

### Adding Raw data

In [7]:
### Learn this from githum issue https://github.com/facebook/Ax/issues/768
for i in range(len(modified_features)):
    ax_client.attach_trial(parameters = {ax_parameters[j]['name']: modified_features.values[i][j] for j in range(9)})
    ax_client.complete_trial(trial_index = i, raw_data = {"f": train_final_label.values[i]})


[INFO 01-31 12:56:56] ax.core.experiment: Attached custom parameterizations [{'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 1.25, 'Time': 60.0, 'Temperature': 25.0}] as trial 0.
[INFO 01-31 12:56:56] ax.service.ax_client: Completed trial 0 with data: {'f': (0.086372, None)}.
[INFO 01-31 12:56:56] ax.core.experiment: Attached custom parameterizations [{'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 2.5, 'Time': 60.0, 'Temperature': 25.0}] as trial 1.
[INFO 01-31 12:56:56] ax.service.ax_client: Completed trial 1 with data: {'f': (0.040938, None)}.
[INFO 01-31 12:56:56] ax.core.experiment: Attached custom parameterizations [{'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 5.0, 'Time': 60.0, 'Temperature': 

### Evaluating model so far (optional) 
We want to do a BO loop in the end, but is always nice to see how the model is doing

In [8]:
### For evaluation purposes you can train the model
#AxClient.fit_model(ax_client)
#render(interact_cross_validation(cv_results, show_context=True,autoset_axis_limits=False))


### Get the new trials! (Finally)

In [18]:
ax_client.get_next_trial() 

#########################
#                       #
#     Mayk's comments   #
#                       #
#########################
'''
Running it directly, gave me this error
ModelFittingError: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.
Is that the problem you showed me yesterday with some points?

Couldn't fix it. Jumped to the dev API
'''

[INFO 01-31 12:06:16] ax.service.ax_client: Generated new trial 51 with parameters {'Dimension1': 12.625996, 'Dimension2': 6.0, 'Dilution(OD)': 0.264996, 'Nano_stock_vol': 0.5, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 6.858187, 'Time': 60.0, 'Temperature': 25.0}.


"\nRunning it directly, gave me this error\nModelFittingError: All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode.\nIs that the problem you showed me yesterday with some points?\n\nCouldn't fix it. Jumped to the dev API\n"

##### [Red]Important! For the purposes of this experiment we need to fix the dimensions (Dimension1 and Dimension2 features) as this cannot be controlled from the beggining. In other words, Farwa synthesizes NPLs, reports tu us the dimensions obtained, and then we get the rest of experimental parameters for the experiment. 

##### For this, we HAVE to use the Developer API (this is now a feature request in Ax.dev) https://github.com/facebook/Ax/issues/1951

In [ ]:
###More visualizations
render(ax_client.get_contour_plot(param_x='Dimension1', param_y='Doping%', metric_name='f'))
render(ax_client.get_contour_plot(param_x='Dimension1', param_y='Doping%', metric_name='f'))
render(plot_slice(ax_client.generation_strategy.model, 'Dilution(OD)', metric_name='f'))
render(plot_slice(ax_client.generation_strategy.model, 'Doping%', metric_name='f'))
render(plot_slice(ax_client.generation_strategy.model, 'Dimension2', metric_name='f'))
render(plot_slice(ax_client.generation_strategy.model, 'Dimension1', metric_name='f'))
cv_results = cross_validate(ax_client.generation_strategy.model,)
render(interact_cross_validation(cv_results, show_context=True,autoset_axis_limits=False))

## Using the Developer API

### Specify The Loop Ingredients

In [8]:
#optimization_config =  {"f":ObjectiveProperties(minimize=False)}
objective_metric = Metric(name="f", lower_is_better=None)  

### Not sure why i have to do this still. It seems to be a way of handling data communitation between the model and the experiment
class MyRunner(Runner):
    def run(self, trial):
        trial_metadata = {"name": str(trial.index)}
        return trial_metadata

# Define the search space based on the ax_parameters
# Run the cell of search_space above to have the ax_parameters. Is just easier at this point haha
search_space = SearchSpace(
    parameters=[
        RangeParameter(
            name=param["name"], 
            parameter_type=ParameterType.FLOAT, 
            lower=float(param["bounds"][0]), 
            upper=float(param["bounds"][1])
        )
        if param["type"] == "range" else
        ChoiceParameter(
            name=param["name"],
            values=param["values"],
            parameter_type=ParameterType.FLOAT
        )
        for param in ax_parameters
    ]
)

# Define the experiment with the search space
#the optimization configuration (the objective and metrics)
experiment = Experiment(
    #name="f",
    search_space=search_space,
    optimization_config=OptimizationConfig(objective=Objective(objective_metric, minimize=False)),
    runner=MyRunner(),
)

experiment.warm_start_from_old_experiment(ax_client.generation_strategy.experiment)


/Users/jorgemedina/miniforge3/envs/DOEv2/lib/python3.10/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "Nano_stock_vol". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction.

/Users/jorgemedina/miniforge3/envs/DOEv2/lib/python3.10/site-packages/ax/core/parameter.py:517: UserWarning:

`sort_values` is not specified for `ChoiceParameter` "Nano_stock_vol". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

/Users/jorgemedina/miniforge3/envs/DOEv2/lib/python3.10/site-packages/ax/core/parameter.py:517: UserWarning:

`is_ordered` is not specified for `ChoiceParameter` "Ag_acet_conc". Defaulting to `True` for parameters of `ParameterType` FLOAT. To override this behavior (or avoid this warning),

[Trial(experiment_name='None', index=0, status=TrialStatus.COMPLETED, arm=Arm(name='0_0', parameters={'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 1.25, 'Time': 60.0, 'Temperature': 25.0})),
 Trial(experiment_name='None', index=1, status=TrialStatus.COMPLETED, arm=Arm(name='1_0', parameters={'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 2.5, 'Time': 60.0, 'Temperature': 25.0})),
 Trial(experiment_name='None', index=2, status=TrialStatus.COMPLETED, arm=Arm(name='2_0', parameters={'Dimension1': 18.8, 'Dimension2': 14.04, 'Dilution(OD)': 0.175, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 7.8, 'Doping%': 5.0, 'Time': 60.0, 'Temperature': 25.0})),
 Trial(experiment_name='None', index=3, status=TrialStatus.COMPLETED, arm=Arm(name='3_0', parameters={'Dimension1': 18.8, 'Dimension2':

### Data


Option 1: If you already went through the Service API part.


In [11]:
ax_client.generation_strategy.experiment.fetch_data().df
data = ax_client.generation_strategy.experiment.fetch_data() 
data.df

NameError: name 'ax_client' is not defined

Option 2: If you are only using the Developer API you can add the data manually (learned from https://github.com/facebook/Ax/issues/768)

In [14]:
for i in range(len(modified_features)):
    #parameters
    start_params = {ax_parameters[j]['name']: modified_features.values[i][j] for j in range(9)} #change this accordingly, this is for 9 features ordered as in the search_space defined above (you better run that cell!)
    start_data = Data(
        df=pd.DataFrame.from_records([{"arm_name": f"{i}_0",
                                       "metric_name":"f",
                                       "mean": train_final_label.values[i],
                                       "sem": 0.0,
                                       "trial_index": i}]),
    )
    trial = experiment.new_trial()
    trial.add_arm(Arm(name=f"{i}_0", parameters=start_params))
    experiment.attach_data(start_data)
    trial.run().complete()
    
experiment.fetch_data().df.tail()

,arm_name,metric_name,mean,sem,trial_index
46,46_0,f,0.007014,0.0,46
47,47_0,f,0.006404,0.0,47
48,48_0,f,0.039425,0.0,48
49,49_0,f,0.019595,0.0,49
50,50_0,f,0.112990,0.0,50


In [ ]:
#########################
#                       #
#     Mayk's comments   #
#                       #
#########################

#start_data.true_df
#start_data has only one point. Should that be the case?

#ax_client.generation_strategy.experiment.fetch_data().df

#########################
#                       #
#     Jorge's comments  #
#                       #
#########################
## It iterates through the whole data points, so the experiment ends up with all the data points

### Models and Acquisition Function

In [9]:
model_bridge_with_GPEI = Models.BOTORCH_MODULAR(
    experiment=experiment,
    #data=experiment.fetch_data(),                     ###if option 1 for Data uncomment this line
    surrogate=Surrogate(ExactGPModel,mll_class=LeaveOneOutPseudoLikelihood),
                                   # ),  # Optional, will use default if unspecified
    botorch_acqf_class=qExpectedImprovement,  # Optional, will use default if unspecified
    #transforms=[StandardizeY(), UnitX()],    # Optional, will use default if unspecified (see Transforms documentation)
)

#########################
#                       #
#     Mayk's comments   #
#                       #
#########################
# `experiment` has only one point. Might that be the problem?
# DataRequiredError: `StandardizeY` transform requires non-empty data.
# It works if I uncomment the line with `data=data` in the `model_bridge_with_GPEI` definition
# But it's not working for option 2

DataRequiredError: `StandardizeY` transform requires non-empty data.

### And get my new experiments! 

In [ ]:
### Assuming the NPL synthesized in the lab has Dimension1 = 27.5728 and Dimension2 = 8.1029
### We can generate a new trial with these values fixed, so that the model can optimize the acquisition function for the remaining variables

generator_run = model_bridge_with_GPEI.gen(n=1,fixed_features=ObservationFeatures({'Dimension1':27.5728,'Dimension2':8.1029}))
trial = experiment.new_trial(generator_run=generator_run)

#########################
#                       #
#     Mayk's comments   #
#                       #
#########################
# Maybe print the new point here

In [ ]:
trial.run().complete()

Trial(experiment_name='test_f', index=102, status=TrialStatus.COMPLETED, arm=Arm(name='102_0', parameters={'Dimension1': 27.5728, 'Dimension2': 8.1029, 'Dilution(OD)': 0.40492337655113503, 'Nano_stock_vol': 0.25, 'Ag_acet_conc': 0.006, 'Ag_acet_mass': 14.0, 'Doping%': 10.0, 'Time': 60.0, 'Temperature': 50.0}))

In [ ]:

#########################
#                       #
#     Mayk's comments   #
#                       #
#########################
'''
A few considerations:
- Are both approaches (service or dev API) supposed to be independent?
    One can only use the dev api is initialize the ax.Client, which is defined in the service API section.

- The service API didn't work for me. Got that error where the GP couldn't be fitted.
- However, if I load the data using the code in the service API and use the dev API to run, it works.
    While just using the dev API to start an experiment without the "old data", doesn't work. I got this error with the StandardizeY.

Which API do you think is the best? Maybe we should just stick with one of them for the project and use that one only.
'''

#########################
#                       #
# Jorge's comments      #
#                       #
#########################
''' 
Both APIs are equivalent in this setting. I put both, cause the Service API is more user-friendly.
And also so this notebook can be used as a sort of tutorial for Ax too. 

Im curious that the Developer API worked, yesterday i had the issue with both of them.
----> I figure it out, one of them uses a different mll. The developer one, and it doesnt crash (the second one is supposed to be more stable)
'''